![](https://miro.medium.com/max/1059/1*ZAT-ZZQaX80h-Hl1eJr7bA.png)

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [0]:
#!pip install pydrive

In [95]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [77]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: model-stackoverflow30sep2.h5, id: 1kg0tskUeHUEAcvSmiFkq06HLfwuqmDFN
title: model-stackoverflow30sep1.h5, id: 1-3VXak_lf6J64tTFbCAPV0dp7zv_mmuC
title: model-stackoverflow30sep.h5, id: 1-1M2_edH6aZZ8n0v71GknMDmxbjdWoEp
title: model1-stackoverflowconv30sept.h5, id: 1MTFLH0-olVy9TgvBzqIIhry6F1hhxyze
title: model1-stackoverflowconv.h5, id: 1--DTzgSG5F42zyJvWMFG0K3gsCq_yTYX
title: model-stackoverflow.h5, id: 1bnbooJnEcYUwu0MX43wbGR786cJvEY9w
title: Questions.csv, id: 1i1w8_iIWJRJQ5mJgKHdfwTMJK48wvD3o
title: Tags.csv, id: 1ISMH8qwIa2U-XThdKItf5ulXiYOuYXDh
title: StackOverflow, id: 1zyeiLFYyxKQFGclKy_bZkr0dvkJSPjiI
title: Resume.pdf, id: 1ZfFJW9JGwAbRrUXT8VnZTmpGanLfJu0R
title: cat_dog_sub.zip, id: 1UNlQHo3wllhHls5w_Tq-9ZCg4wRIKawk
title: googlespydering.py, id: 1MELP9JDtMvq5bi4iEhRTJ7s6IcpPNlSV
title: webscraping, id: 1nINpL3MXFrAsHVbKJVKRi8YRQFP-TtVz
title: 8th sem Project, id: 19gldHM3nSSnncBX-40fdrZtIdxFUHeVN
title: Saifurrahman.pdf, id: 1aNdla7WSPX3jrY6BNRqjD-g9whHjEzo1
title: NATS

In [0]:
downloaded = drive.CreateFile({'id': '1ISMH8qwIa2U-XThdKItf5ulXiYOuYXDh'})
downloaded.GetContentFile('Tags.csv')

In [0]:
downloaded = drive.CreateFile({'id':'1kg0tskUeHUEAcvSmiFkq06HLfwuqmDFN'})
downloaded.GetContentFile('model-stackoverflow30sep2.h5')

In [0]:
downloaded = drive.CreateFile({'id': '1i1w8_iIWJRJQ5mJgKHdfwTMJK48wvD3o'})
downloaded.GetContentFile('Questions.csv')

In [80]:
import os
os.listdir()

['.config',
 'adc.json',
 'Questions.csv',
 'Tags.csv',
 'gdrive',
 'model-stackoverflow30sep2.h5',
 'sample_data']

## Importing the required Libraries

In [0]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.cluster import KMeans

from sklearn.utils import class_weight


from scipy.sparse import hstack

warnings.filterwarnings("ignore")
plt.style.use('bmh')
%matplotlib inline

In [0]:
labels = pd.read_csv('Tags.csv')

In [0]:
df_questions = pd.read_csv("Questions.csv",encoding='latin1')

In [13]:
labels.shape, df_questions.shape

((3750994, 2), (1264216, 7))

In [14]:
labels.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [0]:
x = list(labels['Tag'].value_counts().head(10))

In [0]:
labels['Tag'] = labels['Tag'].astype(str)

# grouping tags with id 

In [0]:
grouped_tags = labels.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))

In [0]:
grouped_tags = grouped_tags.reset_index()

In [19]:
grouped_tags.head()

,Id,Tag
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction


In [0]:
grouped_tags_final = pd.DataFrame({'Id':grouped_tags.Id, 'Tags':grouped_tags.Tag})

In [21]:
grouped_tags_final.head(5)

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction


## drop irrelevant columns

In [0]:
df_questions.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)

In [0]:
df_questions = df_questions.merge(grouped_tags_final, on='Id')

In [24]:
df_questions.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction


In [0]:
new_df = df_questions[df_questions['Score']>5]

In [26]:
new_df.shape

(72950, 5)

### Remove duplicate rows

In [27]:
print('Dupplicate entries: {}'.format(new_df.duplicated().sum()))
new_df.drop_duplicates(inplace = True)

Dupplicate entries: 0


In [0]:
new_df.drop(columns=['Id', 'Score'], inplace=True)

In [0]:
new_df['Tags'] = new_df['Tags'].apply(lambda x: x.split())

In [30]:
new_df.head()

,Title,Body,Tags
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [0]:
all_tags = [item for sublist in new_df['Tags'].values for item in sublist]

In [32]:
len(all_tags)

224129

In [33]:
my_set = set(all_tags)
unique_tags = list(my_set)
len(unique_tags)

14883

### Reducing the problem to the most common tags in the dataset

In [0]:
import nltk

In [0]:
flat_list = [item for sublist in new_df['Tags'].values for item in sublist]

keywords = nltk.FreqDist(flat_list)

keywords = nltk.FreqDist(keywords)

frequencies_words = keywords.most_common(100)
tags_features = [word[0] for word in frequencies_words]

In [0]:
#tags_features

In [0]:
#fig, ax = plt.subplots(figsize=(15, 10))
#keywords.plot(100, cumulative=False)

In [0]:
def most_common(tags):
    tags_filtered = []
    for i in range(0, len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [0]:
new_df['Tags'] = new_df['Tags'].apply(lambda x: most_common(x))
new_df['Tags'] = new_df['Tags'].apply(lambda x: x if len(x)>0 else None)

In [40]:
new_df.shape

(72950, 3)

In [41]:
new_df.head()

,Title,Body,Tags
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,None
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,[svn]
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,[algorithm]
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"


In [0]:
new_df.dropna(subset=['Tags'], inplace=True)

In [43]:
new_df.shape

(63167, 3)



## In the next two columns: Body and Title, I'll use lots of text processing:

    Removing html format
    Lowering text
    Transforming abbreviations
    Removing punctuation (but keeping words like c# since it's the most popular tag)
    Lemmatizing words
    Removing stop words

In [0]:
from bs4 import BeautifulSoup
import re

In [0]:
new_df['Body'] = new_df['Body'].apply(lambda x: BeautifulSoup(x).get_text())

In [0]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [0]:
new_df['Body'] = new_df['Body'].apply(lambda x: clean_text(x))

In [0]:
token = ToktokTokenizer()

In [0]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [0]:
def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

In [0]:
def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in tags_features:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

In [0]:
new_df['Body'] = new_df['Body'].apply(lambda x: clean_punct(x)) 

In [0]:
import nltk
from nltk.corpus import stopwords

In [54]:
nltk.download('stopwords')
lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Lemmatization and Stopwords

In [0]:
def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    
    stop_words = set(stopwords.words("english"))
    
    words=token.tokenize(text)
    
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))

In [56]:
nltk.download('wordnet')
new_df['Body'] = new_df['Body'].apply(lambda x: lemitizeWords(x)) 
new_df['Body'] = new_df['Body'].apply(lambda x: stopWordsRemove(x)) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## DataPreparation for Title

In [0]:
new_df['Title'] = new_df['Title'].apply(lambda x: str(x))
new_df['Title'] = new_df['Title'].apply(lambda x: clean_text(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: clean_punct(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: lemitizeWords(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: stopWordsRemove(x)) 

In [58]:
new_df.head()

,Title,Body,Tags
1,good branch merge tutorials tortoisesvn,really good tutorials explain branch merge apa...,[svn]
2,asp.net site map,anyone get experience create sql-based asp.net...,"[sql, asp.net]"
3,function create color wheel,something pseudo-solved many time never quite ...,[algorithm]
4,add script functionality .net applications,little game write c use database back-end trad...,"[c#, .net]"
5,use nest class case,work collection class use video playback recor...,"[c++, oop, class]"


## Data preparation for tags

In [0]:
X1 = new_df['Body']+' '+new_df['Title']
X2 = new_df['Title']
y = new_df['Tags']

In [60]:
X1[4]

'little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call game make thing maintainablemoddable would like class card source code database essentially compile first use addchange card add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies possible register class source file instantiate etc icard card current new mygamecardlibrarycard056 card current onenterplay ref currentgamestate language c# extra bonus possible write script .net language add script functionality .net applications'

In [61]:
X2[4]

'add script functionality .net applications'

In [0]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [63]:
X1.shape

(63167,)

## Creating Features from Text

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [0]:
maxlen = 180

In [0]:

max_words = 5000
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X1)
#tokenizer.fit_on_texts(df_questions.Title)
#tokenizer.fit_on_texts(df_questions.Body)

def get_features(text_series):
    """
    transforms text data to feature_vectors that can be used in the ml model.
    tokenizer must be available.
    """
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, maxlen=maxlen)


In [0]:
def prediction_to_label(prediction):
    tag_prob = [(tags_features[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=True))

In [0]:
x1 = get_features(X1)
#x2 = get_features(X2)

In [69]:
x1.shape

(63167, 180)

In [0]:
#X_tfidf = np.concatenate((x1, x2), axis=1)

In [0]:
#X_tfidf.shape

In [70]:
y_bin.shape

(63167, 100)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x1, y_bin, test_size = 0.1, 
                                                    random_state = 0) 


## Evaluation metrics Average Jacard loss

In [0]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")    

In [73]:
len(flat_list), len(set(flat_list))

(224129, 14883)

In [0]:
#[(x,flat_list.count(x)) for x in tags_features]

## Calculating weights for labels

In [0]:
class_weight = class_weight.compute_class_weight('balanced', tags_features, [i for i in flat_list if i in tags_features])
class_weight_dict = dict(enumerate(class_weight))


## Keras Model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.models import Model

filter_length = 300
max_len = 180

def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 30, input_length=max_len)(inputs)
    layer = LSTM(300)(layer)
    layer = Dense(500, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(200, name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(100, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model
  
model = RNN()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()



In [93]:
import time

filepath = '/content/gdrive/My Drive/model-stackoverflow.h5'
callbacks = [
    ReduceLROnPlateau(), 
    EarlyStopping(patience=4), 
    ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                    save_weights_only=True, 
                    mode='auto', period=10, save_best_only=False)
            ]

history = model.fit(X_train, y_train,
                    class_weight=class_weight,
                    epochs=12,
                    batch_size=30,
                    validation_split=0.1,
                    callbacks=callbacks)

Train on 51165 samples, validate on 5685 samples
Epoch 1/12
51165/51165 [==============================] - 1189s 23ms/step - loss: 0.0773 - acc: 0.9829 - val_loss: 0.0662 - val_acc: 0.9847
Epoch 2/12
51165/51165 [==============================] - 1346s 26ms/step - loss: 0.0559 - acc: 0.9856 - val_loss: 0.0447 - val_acc: 0.9870
Epoch 3/12
51165/51165 [==============================] - 1225s 24ms/step - loss: 0.0412 - acc: 0.9879 - val_loss: 0.0364 - val_acc: 0.9887
Epoch 4/12
51165/51165 [==============================] - 1267s 25ms/step - loss: 0.0351 - acc: 0.9892 - val_loss: 0.0333 - val_acc: 0.9900
Epoch 5/12
51165/51165 [==============================] - 1155s 23ms/step - loss: 0.0319 - acc: 0.9901 - val_loss: 0.0312 - val_acc: 0.9903
Epoch 6/12
51165/51165 [==============================] - 1206s 24ms/step - loss: 0.0297 - acc: 0.9906 - val_loss: 0.0308 - val_acc: 0.9904
Epoch 7/12
51165/51165 [==============================] - 1166s 23ms/step - loss: 0.0280 - acc: 0.9910 - val_lo

OSError: ignored

In [0]:
model.save_weights("/content/gdrive/My Drive/model-stackoverflow3oct.h5")

In [0]:
#model.load_weights('model-stackoverflow3oct.h5')

In [0]:
#import keras

#cnn_model = keras.models.load_model('model-stackoverflow.h5')
#metrics = cnn_model.evaluate(x_test, y_test)
#print("{}: {}".format(model.metrics_names[0], metrics[0]))
#print("{}: {}".format(model.metrics_names[1], metrics[1]))

In [99]:


#cnn_model = keras.models.load_model('model-stackoverflow.h5')
metrics = model.evaluate(X_test, y_test)
print("{}: {}".format(model.metrics_names[0], metrics[0]))
print("{}: {}".format(model.metrics_names[1], metrics[1]))

6317/6317 [==============================] - 23s 4ms/step
loss: 0.028800970486540553
acc: 0.9908105196106636


## Test data from stackoverflow api

In [100]:
import requests
import random
url='https://api.stackexchange.com/2.2/questions?page=1&pagesize=1&order=desc&sort=activity&tagged=python; sql&site=stackoverflow'

data = requests.get(url).json()
print(data)
item= random.choice(data.get('items'))
print(item)
#q=strip_html_tags(item.get('body'))
#print(q)
print(item.get('title'))

{'items': [{'tags': ['python', 'sql', 'django'], 'owner': {'reputation': 8, 'user_id': 11044656, 'user_type': 'registered', 'profile_image': 'https://lh5.googleusercontent.com/-r78ICAxKUD0/AAAAAAAAAAI/AAAAAAAAAGM/LmVSu_aO63Q/photo.jpg?sz=128', 'display_name': 'SAMUEL Tuyizere', 'link': 'https://stackoverflow.com/users/11044656/samuel-tuyizere'}, 'is_answered': True, 'view_count': 21, 'answer_count': 2, 'score': 0, 'last_activity_date': 1570115295, 'creation_date': 1570115140, 'question_id': 58221951, 'link': 'https://stackoverflow.com/questions/58221951/how-do-i-count-objects-that-has-an-id-below-the-current-id-in-django', 'title': 'How do i count objects that has an id below the current id in django'}], 'has_more': True, 'quota_max': 300, 'quota_remaining': 299}
{'tags': ['python', 'sql', 'django'], 'owner': {'reputation': 8, 'user_id': 11044656, 'user_type': 'registered', 'profile_image': 'https://lh5.googleusercontent.com/-r78ICAxKUD0/AAAAAAAAAAI/AAAAAAAAAGM/LmVSu_aO63Q/photo.jpg?sz

In [0]:
def strip_html_tags(body):
    regex = re.compile('<.*?>')
    return re.sub(regex, '', body)

## Predicting on Test split

In [0]:
new_df.reset_index(inplace=True)

In [103]:
X_test.shape

(6317, 180)

In [0]:
s=563
z = X_test[s].reshape(1 ,180)

In [0]:
l = model.predict(z).reshape(100, )
p1 = prediction_to_label(l)
df = pd.DataFrame()
df['label'] = p1.keys()
df['probability'] = p1.values()

In [106]:
#print(y_test[s],tags_features[list(y_test[s]).index(1)])
c=[i for i,j in enumerate (y_test[s]) if j==1]
print([tags_features[i] for i in c])
df.head(10)

['algorithm', 'templates']


,label,probability
0,templates,0.983637
1,mongodb,0.190881
2,ruby-on-rails,0.074304
3,sql-server,0.030235
4,algorithm,0.025115
5,swing,0.024154
6,svn,0.022318
7,asp.net-mvc,0.016334
8,function,0.015413
9,c,0.008426


In [90]:
np.where(df['label']=='algorithm')

(array([32]),)

In [0]:
import math

def softmax(z):
    return [1 / (1+math.exp(-n)) for n in z]